In [1]:
META_DIR="/kaggle/repo/output/infer/vlsp2023emo"

!ls $META_DIR/*/ -d

/kaggle/repo/output/infer/vlsp2023emo/11_26_adapt_frozen_emo_enc_only/
/kaggle/repo/output/infer/vlsp2023emo/11_28_adapt_freeze_emo_end_mfa/
/kaggle/repo/output/infer/vlsp2023emo/11_29_emofs2mas_fixed/
/kaggle/repo/output/infer/vlsp2023emo/12-01-freeze-all-enc/
/kaggle/repo/output/infer/vlsp2023emo/12-01-freeze-all-enc-GiangOi2/
/kaggle/repo/output/infer/vlsp2023emo/12-01-freeze-emo_enc+mha+text_enc/
/kaggle/repo/output/infer/vlsp2023emo/12-05-EXP14-emofs2+shuffle_ref_mel+with_aug-120k/
/kaggle/repo/output/infer/vlsp2023emo/12-05-EXP14-emofs2+shuffle_ref_mel+without_aug-120k/
/kaggle/repo/output/infer/vlsp2023emo/12-05-EXP14-emofs2+shuffle_ref_mel+without_aug-60k/
/kaggle/repo/output/infer/vlsp2023emo/12-05-EXP15-multi-speaker-baseline-50k/
/kaggle/repo/output/infer/vlsp2023emo/12-07-EXP17_multi_speaker_move_emo_enc_819177ad-50k/
/kaggle/repo/output/infer/vlsp2023emo/12-13-EXP15-multi-speaker-baseline-150k/
/kaggle/repo/output/infer/vlsp2023emo/12-13-EXP17_multi_speaker_move_emo_enc_81

In [3]:
meta_dic = {
    "finetune+giangoi": f"{META_DIR}/12-01-freeze-all-enc-GiangOi2/",
    "multi_spk+giangoi": f"{META_DIR}/12-13-EXP15-multi-speaker-baseline-150k/GiangOi2/",
    "multi_spk+move_emo+giangoi": f"{META_DIR}/12-13-EXP17_multi_speaker_move_emo_enc_819177ad-150k/GiangOi2/",
    "ground_truth+giangoi": f"{META_DIR}/GroundTruth2" # fake
}
meta_dic

{'finetune+giangoi': '/kaggle/repo/output/infer/vlsp2023emo/12-01-freeze-all-enc-GiangOi2/',
 'multi_spk+giangoi': '/kaggle/repo/output/infer/vlsp2023emo/12-13-EXP15-multi-speaker-baseline-150k/GiangOi2/',
 'multi_spk+move_emo+giangoi': '/kaggle/repo/output/infer/vlsp2023emo/12-13-EXP17_multi_speaker_move_emo_enc_819177ad-150k/GiangOi2/',
 'ground_truth+giangoi': '/kaggle/repo/output/infer/vlsp2023emo/GroundTruth2'}

In [31]:
import os
import pandas as pd

def get_files(directory, meta_name="finetune+giangoi"):
    spk_name = meta_name.split("+")[-1].upper()
    assert spk_name in ["GIANGOI", "LMH", "TTH"]
    
    files = os.listdir(directory)
    fpaths = [os.path.join(directory, f) for f in files]
    names = [f.split(".")[0].split('-') for f in files]
    fids = [n[0] for n in names]
    if 'ground_truth' in meta_name:
        new_fids = []
        for fid in fids:
            new_fid = fid.split('_')
            new_fid.pop(-1)
            new_fid = '_'.join(new_fid)
            new_fids.append(new_fid)
        fids = new_fids
    if len(names[0]) == 3:
        femo = [n[2] for n in names]
    else: 
        femo = [n.split('.')[0].split('_')[-1] for n in files]
    df = pd.DataFrame({"fid": fids, "femo": femo})
    df['fspk'] = spk_name
    df['fpath'] = fpaths
    
    return df

df = get_files(meta_dic["ground_truth+giangoi"], 'ground_truth+giangoi')
df

,fid,femo,fspk,fpath
0,neu_val_000214,neutral,GIANGOI,/kaggle/repo/output/infer/vlsp2023emo/GroundTr...
1,neu_val_005028,neutral,GIANGOI,/kaggle/repo/output/infer/vlsp2023emo/GroundTr...
2,emo_val_001694,neutral,GIANGOI,/kaggle/repo/output/infer/vlsp2023emo/GroundTr...
3,emo_val_004700,angry,GIANGOI,/kaggle/repo/output/infer/vlsp2023emo/GroundTr...
4,emo_val_005212,sad,GIANGOI,/kaggle/repo/output/infer/vlsp2023emo/GroundTr...
5,emo_val_001627,happy,GIANGOI,/kaggle/repo/output/infer/vlsp2023emo/GroundTr...
6,emo_val_001120,surprise,GIANGOI,/kaggle/repo/output/infer/vlsp2023emo/GroundTr...


In [48]:
gt_df = get_files(meta_dic["ground_truth+giangoi"], 'ground_truth+giangoi')
gt_df['key'] = gt_df['fid'] + "-" + gt_df['femo']
gt_df = gt_df[['key', 'fpath']]
gt_df.columns = ['key', 'ground_truth']

for key in meta_dic:
    if key == "ground_truth+giangoi":
        continue
    df = get_files(meta_dic[key], key)
    df['key'] = df['fid'] + "-" + df['femo']
    df = df[['key', 'fpath']]
    df.columns = ['key', key]
    # join by key
    gt_df = gt_df.merge(df, on='key', how='left')
    

In [50]:
# gt_df = gt_df[['key', 'fpath', 'finetune+giangoi', 'multi_spk+giangoi', 'multi_spk+move_emo+giangoi']]
gt_df

,key,ground_truth,finetune+giangoi,multi_spk+giangoi,multi_spk+move_emo+giangoi
0,neu_val_000214-neutral,/kaggle/repo/output/infer/vlsp2023emo/GroundTr...,/kaggle/repo/output/infer/vlsp2023emo/12-01-fr...,/kaggle/repo/output/infer/vlsp2023emo/12-13-EX...,/kaggle/repo/output/infer/vlsp2023emo/12-13-EX...
1,neu_val_005028-neutral,/kaggle/repo/output/infer/vlsp2023emo/GroundTr...,/kaggle/repo/output/infer/vlsp2023emo/12-01-fr...,/kaggle/repo/output/infer/vlsp2023emo/12-13-EX...,/kaggle/repo/output/infer/vlsp2023emo/12-13-EX...
2,emo_val_001694-neutral,/kaggle/repo/output/infer/vlsp2023emo/GroundTr...,/kaggle/repo/output/infer/vlsp2023emo/12-01-fr...,/kaggle/repo/output/infer/vlsp2023emo/12-13-EX...,/kaggle/repo/output/infer/vlsp2023emo/12-13-EX...
3,emo_val_004700-angry,/kaggle/repo/output/infer/vlsp2023emo/GroundTr...,/kaggle/repo/output/infer/vlsp2023emo/12-01-fr...,/kaggle/repo/output/infer/vlsp2023emo/12-13-EX...,/kaggle/repo/output/infer/vlsp2023emo/12-13-EX...
4,emo_val_005212-sad,/kaggle/repo/output/infer/vlsp2023emo/GroundTr...,/kaggle/repo/output/infer/vlsp2023emo/12-01-fr...,/kaggle/repo/output/infer/vlsp2023emo/12-13-EX...,/kaggle/repo/output/infer/vlsp2023emo/12-13-EX...
5,emo_val_001627-happy,/kaggle/repo/output/infer/vlsp2023emo/GroundTr...,/kaggle/repo/output/infer/vlsp2023emo/12-01-fr...,/kaggle/repo/output/infer/vlsp2023emo/12-13-EX...,/kaggle/repo/output/infer/vlsp2023emo/12-13-EX...
6,emo_val_001120-surprise,/kaggle/repo/output/infer/vlsp2023emo/GroundTr...,/kaggle/repo/output/infer/vlsp2023emo/12-01-fr...,/kaggle/repo/output/infer/vlsp2023emo/12-13-EX...,/kaggle/repo/output/infer/vlsp2023emo/12-13-EX...


In [59]:
# copy all files to wav dir
import shutil


for key in meta_dic:
    directory = meta_dic[key]
    shutil.copytree(directory, "./axy_wavs/" + key)

In [70]:
# create questions
import random


questions = []
iii = 1

other_cols = list(gt_df.columns)[2:]
gt_folder_name = "ground_truth+giangoi"
for idx, item in gt_df.iterrows():
    key, gt, *others = item
    
    # loop throught all pairs in others
    for i in range(len(others)):
        for j in range(i+1, len(others)):
            idxs = [i, j]
            random.shuffle(idxs)
            i, j = idxs
            q = {
                "name": f"q{iii}",
                "audio_A": os.path.join("axy_wavs", gt_folder_name, gt.split("/")[-1]),
                "audio_X": os.path.join("axy_wavs", other_cols[i], others[i].split("/")[-1]),
                "audio_Y": os.path.join("axy_wavs", other_cols[j], others[j].split("/")[-1]),
                "title": iii,
            }
            questions.append(q)
            print(q)
            iii += 1
            # break
        # break
    

{'name': 'q1', 'audio_A': 'axy_wavs/ground_truth+giangoi/neu_val_000214_neutral.wav', 'audio_X': 'axy_wavs/finetune+giangoi/neu_val_000214-emofs2_158000-neutral.wav', 'audio_Y': 'axy_wavs/multi_spk+giangoi/neu_val_000214-emofs2_150000-neutral.wav', 'title': 1}
{'name': 'q2', 'audio_A': 'axy_wavs/ground_truth+giangoi/neu_val_000214_neutral.wav', 'audio_X': 'axy_wavs/multi_spk+move_emo+giangoi/neu_val_000214-emofs2_150000-neutral.wav', 'audio_Y': 'axy_wavs/finetune+giangoi/neu_val_000214-emofs2_158000-neutral.wav', 'title': 2}
{'name': 'q3', 'audio_A': 'axy_wavs/ground_truth+giangoi/neu_val_000214_neutral.wav', 'audio_X': 'axy_wavs/multi_spk+move_emo+giangoi/neu_val_000214-emofs2_150000-neutral.wav', 'audio_Y': 'axy_wavs/multi_spk+giangoi/neu_val_000214-emofs2_150000-neutral.wav', 'title': 3}
{'name': 'q4', 'audio_A': 'axy_wavs/ground_truth+giangoi/neu_val_005028_neutral.wav', 'audio_X': 'axy_wavs/multi_spk+giangoi/neu_val_005028-emofs2_150000-neutral.wav', 'audio_Y': 'axy_wavs/finetune+

In [71]:
out_cfg = {
    "page_title": "axy_test_emotion_1", 
    "questions": []
}

random.shuffle(questions)
out_cfg['questions'] = questions

In [72]:
import json
out_audio_dir = "./wavs"
out_cfg_dir = "./configs"


# Write configuration to a JSON file
os.makedirs(out_cfg_dir, exist_ok=True)
with open(os.path.join(out_cfg_dir, "axy_test.json"), "w") as f:
    f.write(json.dumps(out_cfg, indent=4))

# print(f"{args.num_questions} AXY Test configurations created successfully.")
